# Notebook to analyze data

In [1]:
import pandas as pd
import numpy as np

In [2]:
stats = pd.read_csv("/Users/martinasimonotti/asr-assisted-transcription/asr-assisted-transcription/data/output/stats.csv", sep="\t")
stats["module"] = [x.split("_")[1][:-1] for x in stats["Transcript_ID"]]
stats

,num_speakers,Transcript_ID,NomeFile,Audio,Annotatore,Revisore,Tipo,Esperto,Accurato,MinutiEsperienza,...,guessing_spans::2,guessing_spans::3,guessing_spans::4,guessing_spans::5,guessing_spans::6,guessing_spans::7,guessing_spans::8,guessing_spans::9,guessing_spans::10,module
0,2,01_StraParlaA_I,01_StraParlaA_I,Straparla_A,I,-,From-Scratch,sì,NaN,NaN,...,14,22,26,0,0,0,0,0,0,StraParla
1,2,02_StraParlaA_V,02_StraParlaA_V,Straparla_A,V,-,Whisper-Assisted,no,NaN,NaN,...,2,3,3,0,0,0,0,0,0,StraParla
2,2,02_StraParlaA_A,02_StraParlaA_A,Straparla_A,A,-,Whisper-Assisted,sì,NaN,NaN,...,2,2,0,0,0,0,0,0,0,StraParla
3,2,01_ParlaBOA_M,01_ParlaBOA_M,ParlaBO_A,M,-,From-Scratch,no,NaN,NaN,...,3,5,0,0,0,0,0,0,0,ParlaBO
4,2,03_ParlaBOA,03_ParlaBOA,ParlaBO_A,esterno,?,Revised,-,-,-,...,3,6,9,11,13,15,15,0,0,ParlaBO
5,2,01_StraParlaA_L,01_StraParlaA_L,Straparla_A,L,-,From-Scratch,no,NaN,NaN,...,2,0,0,0,0,0,0,0,0,StraParla
6,2,03_ParlaBOB,03_ParlaBOB,ParlaBO_B,esterno,?,Revised,-,-,-,...,1,1,2,2,4,5,5,5,5,ParlaBO
7,3,01_PastiB_V,01_PastiB_V,Pasti_B,V,-,From-Scratch,no,NaN,NaN,...,7,7,0,0,0,0,0,0,0,Pasti
8,2,03_StraParlaA,03_StraParlaA,Straparla_A,NaN,Eleonora,Revised,-,-,-,...,2,2,2,0,0,0,0,0,0,StraParla
9,2,01_ParlaBOB_N,01_ParlaBOB_N,ParlaBO_B,N,-,From-Scratch,no,NaN,NaN,...,4,4,0,0,0,0,0,0,0,ParlaBO


In [ ]:
avg_per_minute_mixed = stats[["module", "Tipo"]+[f"tokens_per_minute::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
# avg_per_minute.loc[("ParlaBO", "Revised"), "tokens_per_minute::8"]
avg_per_minute_tok = avg_per_minute_mixed.round(0).astype(int) # @ Ludovica ho eliminato i decimali ma se servono troviamo il modo di rimetterli
avg_per_minute_tok

tokens_per_minute::0  tokens_per_minute::1  \
module    Tipo                                                           
ParlaBO   From-Scratch                       165                   346   
          Revised                            160                   338   
          Whisper-Assisted                   158                   339   
Pasti     From-Scratch                       165                   379   
          Revised                            171                   394   
          Whisper-Assisted                   167                   372   
StraParla From-Scratch                       157                   351   
          Revised                            159                   366   
          Whisper-Assisted                   170                   358   

                            tokens_per_minute::2  
module    Tipo                                    
ParlaBO   From-Scratch                       510  
          Revised                            494  
          Whisper-Assisted                   502  
Pasti     From-Scratch                       563  
          Revised                            592  
          Whisper-Assisted                   551  
StraParla From-Scratch                       507  
          Revised                            523  
          Whisper-Assisted                   545

Number of tokens per minute

In [ ]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_per_minute_tok.loc[(module, t1), f"tokens_per_minute::{i}"] - avg_per_minute_tok.loc[(module, t2), f"tokens_per_minute::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas

,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-5,-8,-16
1,ParlaBO,Revised - Whisper-Assisted,2,-1,-8
2,Pasti,Revised - From-Scratch,6,15,29
3,Pasti,Revised - Whisper-Assisted,4,22,41
4,StraParla,Revised - From-Scratch,2,15,16
5,StraParla,Revised - Whisper-Assisted,-11,8,-22


Number of transcription units

In [7]:
avg_num_tu_mixed = stats[["module", "Tipo"]+[f"num_tu::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()

In [8]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_num_tu_mixed.loc[(module, t1), f"num_tu::{i}"] - avg_num_tu_mixed.loc[(module, t2), f"num_tu::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,4.67,11.33,20.17
1,ParlaBO,Revised - Whisper-Assisted,7.67,18.33,28.17
2,Pasti,Revised - From-Scratch,6.83,11.00,14.33
3,Pasti,Revised - Whisper-Assisted,1.83,2.00,0.33
4,StraParla,Revised - From-Scratch,-7.47,-16.67,-20.53
5,StraParla,Revised - Whisper-Assisted,4.53,7.73,2.87


## numero di token trascritti



### Isoliamo solo i token linguistici (togliamo shortpauses e token metalinguistici)


EXPERT DELTAS

In [3]:
stats_filtrato = stats[~stats["Esperto"].isin(["no"])] # toglie i non-expert
stats_filtrato

,num_speakers,Transcript_ID,NomeFile,Audio,Annotatore,Revisore,Tipo,Esperto,Accurato,MinutiEsperienza,...,guessing_spans::2,guessing_spans::3,guessing_spans::4,guessing_spans::5,guessing_spans::6,guessing_spans::7,guessing_spans::8,guessing_spans::9,guessing_spans::10,module
0,2,01_StraParlaA_I,01_StraParlaA_I,Straparla_A,I,-,From-Scratch,sì,NaN,NaN,...,14,22,26,0,0,0,0,0,0,StraParla
2,2,02_StraParlaA_A,02_StraParlaA_A,Straparla_A,A,-,Whisper-Assisted,sì,NaN,NaN,...,2,2,0,0,0,0,0,0,0,StraParla
4,2,03_ParlaBOA,03_ParlaBOA,ParlaBO_A,esterno,?,Revised,-,-,-,...,3,6,9,11,13,15,15,0,0,ParlaBO
6,2,03_ParlaBOB,03_ParlaBOB,ParlaBO_B,esterno,?,Revised,-,-,-,...,1,1,2,2,4,5,5,5,5,ParlaBO
8,2,03_StraParlaA,03_StraParlaA,Straparla_A,NaN,Eleonora,Revised,-,-,-,...,2,2,2,0,0,0,0,0,0,StraParla
12,2,03_StraParlaB,03_StraParlaB,Straparla_B,NaN,Eleonora,Revised,-,-,-,...,1,1,0,0,0,0,0,0,0,StraParla
13,3,03_StraParlaC,03_StraParlaC,Straparla_C,NaN,Eleonora,Revised,-,-,-,...,2,0,0,0,0,0,0,0,0,StraParla
16,2,02_ParlaBOA_I,02_ParlaBOA_I,ParlaBO_A,I,-,Whisper-Assisted,sì,NaN,NaN,...,11,14,15,0,0,0,0,0,0,ParlaBO
17,3,02_PastiA_E,02_PastiA_E,Pasti_A,E,-,Whisper-Assisted,sì,NaN,NaN,...,6,8,11,11,11,13,0,0,0,Pasti
18,2,01_StraParlaB_A,01_StraParlaB_A,Straparla_B,A,-,From-Scratch,sì,NaN,NaN,...,10,10,0,0,0,0,0,0,0,StraParla


In [21]:
# TUTTI I TOKEN
# TODO: forse bisogna sostituire gli 0 con N/A nel dataframe?
# columns =
avg_per_minute = stats_filtrato[["module", "Tipo"]+[f"tokens_per_minute::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
# avg_per_minute.loc[("ParlaBO", "Revised"), "tokens_per_minute::8"]

avg_per_minute_ling = stats_filtrato[["module", "Tipo"]+[f"linguistic_tokens_min::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
# avg_per_minute_ling

In [22]:
avg_per_minute = avg_per_minute.round(0).astype(int) # @ Ludovica ho eliminato i decimali ma se servono troviamo il modo di rimetterli
avg_per_minute 


tokens_per_minute::0  tokens_per_minute::1  \
module    Tipo                                                           
ParlaBO   From-Scratch                       161                   342   
          Revised                            160                   338   
          Whisper-Assisted                   156                   332   
Pasti     From-Scratch                       166                   359   
          Revised                            171                   394   
          Whisper-Assisted                   170                   371   
StraParla From-Scratch                       160                   364   
          Revised                            159                   366   
          Whisper-Assisted                   180                   377   

                            tokens_per_minute::2  
module    Tipo                                    
ParlaBO   From-Scratch                       508  
          Revised                            494  
          Whisper-Assisted                   494  
Pasti     From-Scratch                       535  
          Revised                            592  
          Whisper-Assisted                   547  
StraParla From-Scratch                       543  
          Revised                            523  
          Whisper-Assisted                   564

In [23]:
avg_per_minute_ling = avg_per_minute_ling.round(0).astype(int)
avg_per_minute_ling

linguistic_tokens_min::0  \
module    Tipo                                         
ParlaBO   From-Scratch                           154   
          Revised                                155   
          Whisper-Assisted                       148   
Pasti     From-Scratch                           165   
          Revised                                168   
          Whisper-Assisted                       168   
StraParla From-Scratch                           154   
          Revised                                152   
          Whisper-Assisted                       172   

                            linguistic_tokens_min::1  linguistic_tokens_min::2  
module    Tipo                                                                  
ParlaBO   From-Scratch                           333                       498  
          Revised                                334                       488  
          Whisper-Assisted                       319                       474  
Pasti     From-Scratch                           355                       527  
          Revised                                388                       581  
          Whisper-Assisted                       366                       540  
StraParla From-Scratch                           350                       522  
          Revised                                350                       500  
          Whisper-Assisted                       362                       544

| ParlaBO | Delta(FS) | Delta(whi) |

Tokens per minute DELTA

In [24]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_per_minute.loc[(module, t1), f"tokens_per_minute::{i}"] - avg_per_minute.loc[(module, t2), f"tokens_per_minute::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas

,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-1,-4,-14
1,ParlaBO,Revised - Whisper-Assisted,4,6,0
2,Pasti,Revised - From-Scratch,5,35,57
3,Pasti,Revised - Whisper-Assisted,1,23,45
4,StraParla,Revised - From-Scratch,-1,2,-20
5,StraParla,Revised - Whisper-Assisted,-21,-11,-41


Linguistic tokens per minute DELTA

In [26]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_per_minute_ling.loc[(module, t1), f"linguistic_tokens_min::{i}"] - avg_per_minute_ling.loc[(module, t2), f"linguistic_tokens_min::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas

,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,1,1,-10
1,ParlaBO,Revised - Whisper-Assisted,7,15,14
2,Pasti,Revised - From-Scratch,3,33,54
3,Pasti,Revised - Whisper-Assisted,0,22,41
4,StraParla,Revised - From-Scratch,-2,0,-22
5,StraParla,Revised - Whisper-Assisted,-20,-12,-44


Quanto si somigliano le due trascrizioni FS/WA?

In [27]:
avg_num_tu = stats_filtrato[["module", "Tipo"]+[f"num_tu::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_duration_per_minute = stats_filtrato[["module", "Tipo"]+[f"avg_duration_per_min::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_intonation_patterns = stats_filtrato[["module", "Tipo"]+[f"intonation_patterns_min::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_prolongations = stats_filtrato[["module", "Tipo"]+[f"prolongations::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_overlapping_spans = stats_filtrato[["module", "Tipo"]+[f"overlapping_spans::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_differing_pace_spans = stats_filtrato[["module", "Tipo"]+[f"differing_pace_spans::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_differing_volume_spans = stats_filtrato[["module", "Tipo"]+[f"differing_volume_spans::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()

Numero di unità di trascrizione

In [28]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_num_tu.loc[(module, t1), f"num_tu::{i}"] - avg_num_tu.loc[(module, t2), f"num_tu::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,7.00,14.00,23.50
1,ParlaBO,Revised - Whisper-Assisted,12.50,26.00,38.00
2,Pasti,Revised - From-Scratch,-0.50,-1.00,2.00
3,Pasti,Revised - Whisper-Assisted,0.50,2.00,3.00
4,StraParla,Revised - From-Scratch,-1.00,-1.67,-4.33
5,StraParla,Revised - Whisper-Assisted,2.33,2.33,-3.83


Durata media in minuti delle unità di trascrizione

In [81]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_duration_per_minute.loc[(module, t1), f"avg_duration_per_min::{i}"] - avg_duration_per_minute.loc[(module, t2), f"avg_duration_per_min::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) # Rounds decimals


df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-0.52,-0.49,-0.62
1,ParlaBO,Revised - Whisper-Assisted,-1.38,-1.58,-1.60
2,Pasti,Revised - From-Scratch,-0.00,0.05,0.02
3,Pasti,Revised - Whisper-Assisted,-0.05,-0.02,0.00
4,StraParla,Revised - From-Scratch,0.04,-0.08,-0.14
5,StraParla,Revised - Whisper-Assisted,-0.29,-0.08,0.04


Numero di token con pattern intonativo

In [82]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_intonation_patterns.loc[(module, t1), f"intonation_patterns_min::{i}"] - avg_intonation_patterns.loc[(module, t2), f"intonation_patterns_min::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,NaN,-1.0,-2.00
1,ParlaBO,Revised - Whisper-Assisted,-2.00,-5.5,-12.00
2,Pasti,Revised - From-Scratch,-4.50,-1.5,-2.00
3,Pasti,Revised - Whisper-Assisted,-5.50,-7.5,-10.00
4,StraParla,Revised - From-Scratch,-2.67,-5.0,-8.33
5,StraParla,Revised - Whisper-Assisted,1.67,2.5,2.33


Numero di token con allungamenti

In [83]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_prolongations.loc[(module, t1), f"prolongations::{i}"] - avg_prolongations.loc[(module, t2), f"prolongations::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-6.50,-9.50,-19.00
1,ParlaBO,Revised - Whisper-Assisted,-1.00,-1.00,-1.50
2,Pasti,Revised - From-Scratch,1.50,3.00,8.50
3,Pasti,Revised - Whisper-Assisted,-2.50,-10.00,-11.50
4,StraParla,Revised - From-Scratch,-1.33,-7.33,-10.33
5,StraParla,Revised - Whisper-Assisted,0.50,2.33,4.00


Numero di overlap (span)

In [84]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_overlapping_spans.loc[(module, t1), f"overlapping_spans::{i}"] - avg_overlapping_spans.loc[(module, t2), f"overlapping_spans::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,0.50,-1.50,-3.5
1,ParlaBO,Revised - Whisper-Assisted,0.50,18.00,20.0
2,Pasti,Revised - From-Scratch,2.00,9.00,15.5
3,Pasti,Revised - Whisper-Assisted,0.00,4.00,11.5
4,StraParla,Revised - From-Scratch,4.67,4.33,5.0
5,StraParla,Revised - Whisper-Assisted,4.17,5.67,8.0


Numero di span con pace variato

In [85]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_differing_pace_spans.loc[(module, t1), f"differing_pace_spans::{i}"] - avg_differing_pace_spans.loc[(module, t2), f"differing_pace_spans::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,NaN,NaN,NaN
1,ParlaBO,Revised - Whisper-Assisted,0.00,-0.50,0.00
2,Pasti,Revised - From-Scratch,NaN,NaN,3.50
3,Pasti,Revised - Whisper-Assisted,NaN,NaN,NaN
4,StraParla,Revised - From-Scratch,-0.67,-1.33,-2.00
5,StraParla,Revised - Whisper-Assisted,0.00,-0.17,-0.67


Numero di span con volume variato

In [86]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_differing_volume_spans.loc[(module, t1), f"differing_volume_spans::{i}"] - avg_differing_volume_spans.loc[(module, t2), f"differing_volume_spans::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,NaN,NaN,NaN
1,ParlaBO,Revised - Whisper-Assisted,-5.00,-8.50,-14.50
2,Pasti,Revised - From-Scratch,3.50,4.50,9.00
3,Pasti,Revised - Whisper-Assisted,2.50,6.50,12.00
4,StraParla,Revised - From-Scratch,6.33,0.33,-2.33
5,StraParla,Revised - Whisper-Assisted,5.00,1.00,0.67


NON-EXPERT DELTAS

In [7]:
stats_filtrato_non = stats[~stats["Esperto"].isin(["sì"])] # toglie i expert
stats_filtrato_non

,num_speakers,Transcript_ID,NomeFile,Audio,Annotatore,Revisore,Tipo,Esperto,Accurato,MinutiEsperienza,...,guessing_spans::2,guessing_spans::3,guessing_spans::4,guessing_spans::5,guessing_spans::6,guessing_spans::7,guessing_spans::8,guessing_spans::9,guessing_spans::10,module
1,2,02_StraParlaA_V,02_StraParlaA_V,Straparla_A,V,-,Whisper-Assisted,no,NaN,NaN,...,2,3,3,0,0,0,0,0,0,StraParla
3,2,01_ParlaBOA_M,01_ParlaBOA_M,ParlaBO_A,M,-,From-Scratch,no,NaN,NaN,...,3,5,0,0,0,0,0,0,0,ParlaBO
4,2,03_ParlaBOA,03_ParlaBOA,ParlaBO_A,esterno,?,Revised,-,-,-,...,3,6,9,11,13,15,15,0,0,ParlaBO
5,2,01_StraParlaA_L,01_StraParlaA_L,Straparla_A,L,-,From-Scratch,no,NaN,NaN,...,2,0,0,0,0,0,0,0,0,StraParla
6,2,03_ParlaBOB,03_ParlaBOB,ParlaBO_B,esterno,?,Revised,-,-,-,...,1,1,2,2,4,5,5,5,5,ParlaBO
7,3,01_PastiB_V,01_PastiB_V,Pasti_B,V,-,From-Scratch,no,NaN,NaN,...,7,7,0,0,0,0,0,0,0,Pasti
8,2,03_StraParlaA,03_StraParlaA,Straparla_A,NaN,Eleonora,Revised,-,-,-,...,2,2,2,0,0,0,0,0,0,StraParla
9,2,01_ParlaBOB_N,01_ParlaBOB_N,ParlaBO_B,N,-,From-Scratch,no,NaN,NaN,...,4,4,0,0,0,0,0,0,0,ParlaBO
10,4,02_PastiA_M,02_PastiA_M,Pasti_A,M,-,Whisper-Assisted,no,NaN,NaN,...,5,7,9,0,0,0,0,0,0,Pasti
11,3,01_PastiA_R,01_PastiA_R,Pasti_A,R,-,From-Scratch,no,NaN,NaN,...,2,3,0,0,0,0,0,0,0,Pasti


In [30]:
avg_per_minute = stats_filtrato_non[["module", "Tipo"]+[f"tokens_per_minute::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
# avg_per_minute.loc[("ParlaBO", "Revised"), "tokens_per_minute::8"]

avg_per_minute_ling = stats_filtrato_non[["module", "Tipo"]+[f"linguistic_tokens_min::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
# avg_per_minute_ling

In [31]:
avg_per_minute = avg_per_minute.round(0).astype(int) # @ Ludovica ho eliminato i decimali ma se servono troviamo il modo di rimetterli
avg_per_minute 

tokens_per_minute::0  tokens_per_minute::1  \
module    Tipo                                                           
ParlaBO   From-Scratch                       167                   348   
          Revised                            160                   338   
          Whisper-Assisted                   163                   353   
Pasti     From-Scratch                       165                   388   
          Revised                            171                   394   
          Whisper-Assisted                   165                   373   
StraParla From-Scratch                       153                   331   
          Revised                            159                   366   
          Whisper-Assisted                   164                   345   

                            tokens_per_minute::2  
module    Tipo                                    
ParlaBO   From-Scratch                       512  
          Revised                            494  
          Whisper-Assisted                   518  
Pasti     From-Scratch                       578  
          Revised                            592  
          Whisper-Assisted                   554  
StraParla From-Scratch                       452  
          Revised                            523  
          Whisper-Assisted                   532

In [32]:
avg_per_minute_ling = avg_per_minute_ling.round(0).astype(int)
avg_per_minute_ling

linguistic_tokens_min::0  \
module    Tipo                                         
ParlaBO   From-Scratch                           160   
          Revised                                155   
          Whisper-Assisted                       159   
Pasti     From-Scratch                           158   
          Revised                                168   
          Whisper-Assisted                       160   
StraParla From-Scratch                           148   
          Revised                                152   
          Whisper-Assisted                       159   

                            linguistic_tokens_min::1  linguistic_tokens_min::2  
module    Tipo                                                                  
ParlaBO   From-Scratch                           338                       498  
          Revised                                334                       488  
          Whisper-Assisted                       344                       505  
Pasti     From-Scratch                           375                       556  
          Revised                                388                       581  
          Whisper-Assisted                       364                       543  
StraParla From-Scratch                           322                       441  
          Revised                                350                       500  
          Whisper-Assisted                       335                       518

Tokens per minute

In [33]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_per_minute.loc[(module, t1), f"tokens_per_minute::{i}"] - avg_per_minute.loc[(module, t2), f"tokens_per_minute::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas

,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-7,-10,-18
1,ParlaBO,Revised - Whisper-Assisted,-3,-15,-24
2,Pasti,Revised - From-Scratch,6,6,14
3,Pasti,Revised - Whisper-Assisted,6,21,38
4,StraParla,Revised - From-Scratch,6,35,71
5,StraParla,Revised - Whisper-Assisted,-5,21,-9


Linguistic tokens per minute

In [34]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_per_minute_ling.loc[(module, t1), f"linguistic_tokens_min::{i}"] - avg_per_minute_ling.loc[(module, t2), f"linguistic_tokens_min::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas

,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-5,-4,-10
1,ParlaBO,Revised - Whisper-Assisted,-4,-10,-17
2,Pasti,Revised - From-Scratch,10,13,25
3,Pasti,Revised - Whisper-Assisted,8,24,38
4,StraParla,Revised - From-Scratch,4,28,59
5,StraParla,Revised - Whisper-Assisted,-7,15,-18


In [35]:
avg_num_tu = stats_filtrato_non[["module", "Tipo"]+[f"num_tu::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_duration_per_minute = stats_filtrato_non[["module", "Tipo"]+[f"avg_duration_per_min::{i}" for i in range(0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_intonation_patterns = stats_filtrato_non[["module", "Tipo"]+[f"intonation_patterns_min::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_prolongations = stats_filtrato_non[["module", "Tipo"]+[f"prolongations::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_overlapping_spans = stats_filtrato_non[["module", "Tipo"]+[f"overlapping_spans::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_differing_pace_spans = stats_filtrato_non[["module", "Tipo"]+[f"differing_pace_spans::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()
avg_differing_volume_spans = stats_filtrato_non[["module", "Tipo"]+[f"differing_volume_spans::{i}" for i in range (0,3)]].replace(0, np.nan).groupby(["module", "Tipo"]).mean()

Numero unità di trascrizione

In [36]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_num_tu.loc[(module, t1), f"num_tu::{i}"] - avg_num_tu.loc[(module, t2), f"num_tu::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas

,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,3.50,10.00,18.50
1,ParlaBO,Revised - Whisper-Assisted,-2.00,3.00,8.50
2,Pasti,Revised - From-Scratch,10.50,17.00,20.50
3,Pasti,Revised - Whisper-Assisted,2.50,2.00,-1.00
4,StraParla,Revised - From-Scratch,-17.17,-39.17,-44.83
5,StraParla,Revised - Whisper-Assisted,6.00,11.33,7.33


Durata media in minuti delle tu

In [37]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_duration_per_minute.loc[(module, t1), f"avg_duration_per_min::{i}"] - avg_duration_per_minute.loc[(module, t2), f"avg_duration_per_min::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) # Rounds decimals


df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-0.20,-0.28,-0.44
1,ParlaBO,Revised - Whisper-Assisted,0.34,0.05,-0.05
2,Pasti,Revised - From-Scratch,-0.96,-0.87,-0.74
3,Pasti,Revised - Whisper-Assisted,-0.20,-0.09,-0.01
4,StraParla,Revised - From-Scratch,0.69,0.93,0.92
5,StraParla,Revised - Whisper-Assisted,-0.76,-0.46,-0.27


Numero di token con pattern intonativo

In [38]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_intonation_patterns.loc[(module, t1), f"intonation_patterns_min::{i}"] - avg_intonation_patterns.loc[(module, t2), f"intonation_patterns_min::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-1.00,-1.00,-10.50
1,ParlaBO,Revised - Whisper-Assisted,-8.00,-23.00,-37.00
2,Pasti,Revised - From-Scratch,-1.00,0.50,-0.50
3,Pasti,Revised - Whisper-Assisted,1.00,2.50,2.00
4,StraParla,Revised - From-Scratch,-2.83,-7.00,-10.17
5,StraParla,Revised - Whisper-Assisted,-0.83,0.67,0.67


Numero di token con allungamenti

In [39]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_prolongations.loc[(module, t1), f"prolongations::{i}"] - avg_prolongations.loc[(module, t2), f"prolongations::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas

,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-3.50,-9.50,-14.50
1,ParlaBO,Revised - Whisper-Assisted,-1.50,1.50,2.00
2,Pasti,Revised - From-Scratch,0.50,6.50,11.00
3,Pasti,Revised - Whisper-Assisted,1.50,-0.50,-4.50
4,StraParla,Revised - From-Scratch,-2.50,-3.67,-4.00
5,StraParla,Revised - Whisper-Assisted,2.33,6.00,7.67


Numero di overlap (span)

In [40]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_overlapping_spans.loc[(module, t1), f"overlapping_spans::{i}"] - avg_overlapping_spans.loc[(module, t2), f"overlapping_spans::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,2.00,5.50,6.50
1,ParlaBO,Revised - Whisper-Assisted,3.50,3.50,-0.50
2,Pasti,Revised - From-Scratch,3.50,9.50,11.00
3,Pasti,Revised - Whisper-Assisted,5.00,13.50,15.50
4,StraParla,Revised - From-Scratch,0.67,8.17,22.50
5,StraParla,Revised - Whisper-Assisted,9.67,23.00,30.67


Numero di span con pace variato

In [41]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_differing_pace_spans.loc[(module, t1), f"differing_pace_spans::{i}"] - avg_differing_pace_spans.loc[(module, t2), f"differing_pace_spans::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-2.00,-7.00,-8.50
1,ParlaBO,Revised - Whisper-Assisted,0.00,0.50,1.00
2,Pasti,Revised - From-Scratch,NaN,0.00,0.00
3,Pasti,Revised - Whisper-Assisted,3.00,1.50,1.50
4,StraParla,Revised - From-Scratch,0.00,-0.17,-0.17
5,StraParla,Revised - Whisper-Assisted,-0.67,-1.33,-1.33


Numero di span con volume variato

In [42]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 3):
            diff = avg_differing_volume_spans.loc[(module, t1), f"differing_volume_spans::{i}"] - avg_differing_volume_spans.loc[(module, t2), f"differing_volume_spans::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas.iloc[:, 2:] = df_deltas.iloc[:, 2:].round(2) 
df_deltas


,0,1,2,3,4
0,ParlaBO,Revised - From-Scratch,-2.0,-3.5,-6.00
1,ParlaBO,Revised - Whisper-Assisted,-1.0,-4.0,-8.00
2,Pasti,Revised - From-Scratch,-0.5,0.0,-0.50
3,Pasti,Revised - Whisper-Assisted,-0.5,2.5,7.00
4,StraParla,Revised - From-Scratch,7.0,11.0,13.67
5,StraParla,Revised - Whisper-Assisted,5.0,4.5,10.00


2. il numero di minuti trascritti (manca!!)

3. Quanto si somigliano le due trascrizioni FS/WA? In termini di:
   1. il numero di unità di trascrizione
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   2. la durata media in ms delle unità di trascrizione
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   3. la durata media in ms delle unità di trascrizione
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   4. numero di span con overlap
   5. numero di span con pace variato
   6. numero di token con pattern intonativo
   7. numero di span con volume variato
   8. numero di token con allungamenti

4. Calcoliamo Delta-s(Gold, FS), Delta-w(Gold, WA) su:
   1. il numero di unità di trascrizione varia tra fase FS e fase WA?
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   2. la durata media in ms delle unità di trascrizione varia tra fase FS e fase WA?
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   3. la durata media in ms delle unità di trascrizione varia tra fase FS e fase WA?
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   4. numero di span con overlap
   5. numero di span con pace variato
   6. numero di token con pattern intonativo
   7. numero di span con volume variato
   8. numero di token con allungamenti

5. MISMATCH (per esperti e per non esperti) sul livello ortografico
   1. numero di token che mismatchano nelle due condizioni (FS e WA):
      1. li classifichiamo a mano (es. quante congiunzioni? quanti filler? quante shortpauses?...)
      2. valutazione "gravità" del mismatch (presenza/assenza di un token, stesso lemma, token diversi...)
      3. edit distance tra i token mismatchanti per valutare se ci sono typos (es. "parlare" vs "prlare")

6. MATCH
   1. calcolo accuracy (e per l'overlap che è di tipo relazionale UAS-LAS (Unlabeled attachment score e labeled attachment score)) per ogni feature jefferson: differenza tra accuratezza dell'annotatore FS rispetto al Gold e accuratezza dell'annotatore WA rispetto al gold.
   2. media delle accuratezze per file e media delle accuratezze per feature
   3. per i token che matchano a inizio e fine di unità di trascrizione, valutazione della precisione di inizio/fine unità di trascrizione (non si può fare in termini di accuracy perché sono numeri reali)
   4. Relazioni tra tipi di mismatch per le features. Ci sono features che sono più predittive di altre, sulla possibilità che esistano mismatch in altre colonne? Per esempio: avere una bassa accuratezza sui prolungamenti è predittivo rispetto ad avere una bassa accuratezza sul volume della voce?

7. valutazione precisione unità a prescindere dal contenuto
   1. Creazione grafo di corrispondenza delle unità attribuite ad ogni parlante per i vari annotatori.